## 3.1 블록체인 노드 구축

### 1_관련 패키지 import

In [1]:
import hashlib          # hash 함수용 sha256 사용할 라이브러리
import json
from time import time
import random
import requests 
from flask import Flask, request, jsonify

### 2_객체 생성

In [2]:
class Blockchain(object):
    # 1_객체 생성
    def __init__(self):
        self.chain = []                                 # chain에 여러 block들 들어옴
        self.current_transaction = []                   # 임시 transaction 넣어줌
        self.nodes = set()                              # Node 목록을 보관
        self.new_block(previous_hash = 1, proof = 100)  # genesis block 생성

    # 2_해시 암호화 함수
    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()   # hash 라이브러리로 sha256 사용
        return hashlib.sha256(block_string).hexdigest()             
    
    # 3_마지막 블록 호출 함수
    @property
    def last_block(self):
        return self.chain[-1]                           # 체인의 마지막 블록 가져오기!

    # 4_블록 검증 함수
    @staticmethod
    def valid_proof(last_proof, proof):
        guess = str(last_proof + proof).encode()        # 전 proof와 구할 proof 문자열 연결 
        guess_hash = hashlib.sha256(guess).hexdigest()  # 이 hash 값 저장
        return guess_hash[:4] == "0000"                 # 앞 4자리가 0000이면 Trus (알맞은 nonce 값을 찾음)
    
    # 5_PoW(작업 증명) 함수
    def pow(self, last_proof):
        proof = random.randint(-1000000, 1000000)
        while self.valid_proof(last_proof, proof) is False: # valid proof 함수 활용(아래 나옴), 맞을 때까지 반복적으로 검증
            proof = random.randint(-100000, 10000000)
        return proof
    
    # 6_거래 내역 추가 함수
    def new_transaction(self, sender, recipient, amount):
        self.current_transaction.append(
            {
                'sender' : sender, 
                'recipient' : recipient,
                'amount' : amount,
                'timestamp' : time()
            }
        )
        return self.last_block['index'] + 1
    
    # 7_신규 블록 생성 함수
    def new_block(self, proof, previous_hash = None):
        block = {
            'index':len(self.chain)+1,
            'timestamp' : time(),
            'transactions' : self.current_transaction,
            'nonce' : proof,
            'previous_hash' : previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transaction = []
        self.chain.append(block)
        return block
    
    # 8_블록 검증 함수
    def valid_chain(self,chain):
        last_block = chain[0]
        current_index = 1

        while current_index < len(chain):
            block = chain[current_index]
            print('%s' %last_block)
            print('%s' %block)
            print('\n------------\n')
            if block['previous_hash'] != self.hash(last_block):
                return False
            last_block = block
            current_index += 1
            
        return True

### 3_블록체인 객체 기반으로 노드 생성

In [3]:
# 1. 기본정보 설정
blockchain = Blockchain()
my_ip = '0.0.0.0'
my_port = '5000'
node_identifier = 'node_'+my_port
mine_owner = 'master'
mine_profit = 0.1

app = Flask(__name__)

# 2. 블록 정보 호출 함수 full_chain()
@app.route('/chain', methods=['GET'])
def full_chain():
    print('chain info requested!!')
    response = {
        'chain' : blockchain.chain,
        'length' : len(blockchain.chain),
    }
    return jsonify(response), 200

# 3. 신규 거래 추가 함수 new_transaction()
@app.route('/transactions/new', methods = ['POST'])
def new_transaction():
    values = request.get_json()
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount']

    if not all(k in values for k in required):
        return 'missing values', 400
    
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])
    response = {'message': 'Transaction will be added to Block {%s}' % index}

    return jsonify(response), 201

# 4. 채굴 함수 mine()
@app.route('/mine', methods=['GET'])
def mine():
    print("MINING STARTED")
    last_block = blockchain.last_block
    last_proof= last_block['nonce']
    proof = blockchain.pow(last_proof)

    blockchain.new_transaction(
        sender = mine_owner,
        recipient=node_identifier,
        amount = mine_profit # coinbase transaction
    )

    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    print("MINING FINISHED")

    response = {
        'message' : 'new block found',
        'index' : block['index'],
        'transactions' : block['transactions'],
        'nonce' : block['nonce'],
        'previous_hash' : block['previous_hash']
    }

    return jsonify(response), 200

# 노드 시작
if __name__ == '__main__':
    app.run(host=my_ip, port=my_port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.76.28:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Sep/2024 16:53:35] "GET /chain HTTP/1.1" 200 -


chain info requested!!


127.0.0.1 - - [18/Sep/2024 16:54:14] "GET /chain HTTP/1.1" 200 -


chain info requested!!


...

chain info requested!!


127.0.0.1 - - [18/Sep/2024 17:03:14] "GET /chain HTTP/1.1" 200 -


chain info requested!!


127.0.0.1 - - [18/Sep/2024 17:03:36] "GET /chain HTTP/1.1" 200 -


chain info requested!!
